In [21]:
!pip install -q -U kaggle_environments

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [22]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

### Напишем для начала три "статических" бота

In [23]:
%%writefile rock_agent.py
#только камень
def rock_agent(observation, configuration):
    return 0

Writing rock_agent.py


In [24]:
%%writefile paper_agent.py
#только бумага
def paper_agent(observation, configuration):
    return 1

Writing paper_agent.py


In [25]:
%%writefile scissors_agent.py
#только ножницы
def scissors_agent(observation, configuration):
    return 2



Writing scissors_agent.py


### Четыре случайных бота

In [26]:
%%writefile random_agent_rps.py
import random

def random_agent_rps(observation, configuration):
    return random.randrange(0,3)

Writing random_agent_rps.py


In [27]:
%%writefile random_agent_rp.py
import random

def random_agent_rp(observation, configuration):
    return random.randrange(0,2)

Writing random_agent_rp.py


In [28]:
%%writefile random_agent_ps.py
import random

def random_agent_ps(observation, configuration):
    return random.randrange(1,3)

Writing random_agent_ps.py


In [29]:
%%writefile random_agent_rs.py
import random

def random_agent_ps(observation, configuration):
    action  = random.randrange(0,2)
    if action==0:
        return 0
    else:
        return 2

Writing random_agent_rs.py


### Копирование последнего действия противника

In [30]:
%%writefile copy_opponent.py
import random

def copy_opponent(observation, configuration):
    
    if observation.step > 0:
        return observation.lastOpponentAction

    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр. Играть будут все со всеми по одному разу.

In [13]:
""" Данная функция предназначена для проведения соревнования между ботами.
    Имена ботов передаются с помощью списка. 
    Все играют со всеми по 100 партий. 
    Функция возвращает словарь имя_бота-количество побед """

def competition(agent_list):

    competition_dict = {}
    for agent in agent_list:
        competition_dict[agent] = 0

    for i in range(0,len(agent_list)):
        for j in range(i+1, len(agent_list)):   
            score = evaluate("rps", 
                         [agent_list[i], agent_list[j]], 
                         configuration={"episodeSteps": 100,   'tieRewardThreshold': 1})
            if score[0][0] > score[0][1]:
                competition_dict[agent_list[i]] += 1
            elif score[0][0] < score[0][1]:
                competition_dict[agent_list[j]] += 1
            else: #ничья
                pass 

    return competition_dict

Турнир где есть один победитель. Каждый раунд выбывает самый слабый и все повторяется заново. 

In [12]:
agent_list = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'copy_opponent.py', 
                 'random_agent_rps.py', 'random_agent_rp.py', 'random_agent_ps.py', 'random_agent_rs.py']


len_agent_list = len(agent_list) + 1
while len(agent_list)>=1:
    if len(agent_list)==1:
        print('Победитель:',  *agent_list)
        break 
    competition_dict = competition(agent_list)
    print(f"Раунд №{len_agent_list-len(agent_list)}", )
    print(competition_dict)
    print()
    min_key = min(competition_dict.keys(), key=lambda k: competition_dict[k])
    del competition_dict[min_key]
    agent_list = list(competition_dict.keys())

Раунд №1
{'rock_agent.py': 2, 'paper_agent.py': 5, 'scissors_agent.py': 3, 'copy_opponent.py': 0, 'random_agent_rps.py': 4, 'random_agent_rp.py': 3, 'random_agent_ps.py': 4, 'random_agent_rs.py': 4}

Раунд №2
{'rock_agent.py': 3, 'paper_agent.py': 3, 'scissors_agent.py': 4, 'random_agent_rps.py': 1, 'random_agent_rp.py': 3, 'random_agent_ps.py': 2, 'random_agent_rs.py': 4}

Раунд №3
{'rock_agent.py': 3, 'paper_agent.py': 2, 'scissors_agent.py': 3, 'random_agent_rp.py': 2, 'random_agent_ps.py': 2, 'random_agent_rs.py': 3}

Раунд №4
{'rock_agent.py': 3, 'scissors_agent.py': 2, 'random_agent_rp.py': 2, 'random_agent_ps.py': 1, 'random_agent_rs.py': 2}

Раунд №5
{'rock_agent.py': 2, 'scissors_agent.py': 0, 'random_agent_rp.py': 3, 'random_agent_rs.py': 1}

Раунд №6
{'rock_agent.py': 1, 'random_agent_rp.py': 2, 'random_agent_rs.py': 0}

Раунд №7
{'rock_agent.py': 0, 'random_agent_rp.py': 1}

Победитель: random_agent_rp.py
